Optimal Policies with Heterogeneous Agents: Truncation and Transitions
==============

# Computational appendix

**Authors:** *[François Le Grand](https://francois-le-grand.fr/)* and *[Xavier Ragot](http://xavier-ragot.fr/)*

This page presents the implementation and the codes for the paper *[Optimal Policies with Heterogeneous Agents: Truncation and Transitions](http://xavier-ragot.fr/)*. The codes are in Julia (for the steady state) and Matlab and Dynare (for the simulations in the presence of aggregate shocks). 

The steady-state part in Julia relies on five notebooks (alll notebooks can be clicked). This part must be run first since it creates a mat-file called `todynare_Truncation.mat` that is required to launch the Dynare solver.
* [`Main.ipynb`](./Main.html) is the main notebook (the present one). It computes the steady-state and creates the file `todynare_Truncation.mat` for simulating the model with aggregate shocks. It also contains links to the other notebooks. 
* [`Structures.ipynb`](./Structures.html) contains the various structures that we use.
* [`Utils.ipynb`](./Utils.html) contains some useful functions (computing the Gini coefficient, etc.).
* [`SolveAiyagari.ipynb`](./SolveAiyagari.html) solves for the Aiyagari model.
* [`Truncation.ipynb`](./Truncation.html) computes the truncated model. 

The aggregate shock part relies on three files.
* [`Dynare_Truncation.m`](./MATLAB/Dynare_Truncation.html) is the main file for aggregate shocks. It writes a Dynare mod file (called `DyTruncation.mod`) and runs it. This file computes IRFs and second-order moments. It can also simulate a transition path (see the file for further explanation).
* [`post-dynare-stoch-simul.m`](post-dynare-stoch-simul.html) and [`post-dynare-perfect-foresight.m`](post-dynare-perfect-foresight.html) are two files for data post-processing. These files are launched automatically and not meant to be launched on their own.


We now proceed with the `Main` file. We start with packager imports.

# Including packages and other notebooks

We include a number of packages:
* `Parameters`: to use `@unpack`;
* `NBInclude`: to include packages;
* `QuantEcon`: to use the function `rouwenhorst` to discretize the AR(1) package;
* `LinearAlgebra`: to use the `dot  function (inner product);
* `Roots`: to use the `findzero` function (root search);
* `SparseArrays`: to manipulate sparse vectors and sparrse matrices;
* `IterativeSolvers`: to use the function `powm!` (find the largest eigenvalue of a matrix);
* `Plots`: to draw plots;
* `MAT`: to save file under he Matlab .mat format (needed for the interaction with Dynare);
* `FileIO`: loading and saving files;
* `JLD2`: to manipulate JDL2 file formats.

In [1]:
using Parameters       # @unpack
using NBInclude        # including notebooks instead of jl files
using QuantEcon        # rouwenhorst
using LinearAlgebra    # dot
using Roots            # findzero
using SparseArrays     # SparseMatrixCSC
using IterativeSolvers # powm!
using Plots            # Plots
using MAT              # matopen (interaction with Dynare)
using FileIO           # saving file
using JLD2             # manipulating JDL2 file format

We include the following notebooks (their names are clickable):
* [`Structures.ipynb`](./Structures.html) contains the various structures that we use.
* [`Utils.ipynb`](./Utils.html) contains some useful functions (computing the Gini coefficient, etc.).
* [`SolveAiyagari.ipynb`](./SolveAiyagari.html) solves for the Aiyagari model.
* [`Truncation.ipynb`](./Truncation.html) computes the truncated model. 

In [2]:
@nbinclude("Structures.ipynb");
@nbinclude("Utils.ipynb");
@nbinclude("SolveAiyagari.ipynb");
@nbinclude("Truncation.ipynb");

# Computing the truncated model

## The calibration

We use the structure `Economy` of the notebook [`Economy`](./Economy.html) and a special constructor that provides a calibration device. 

For further details on the mechanism of the calibration, please refer to the notebook [`Economy`](./Economy.html).

In [3]:
economy_Tt = Economy(
    KsY=10.26,
    α=0.36,
    δ=0.025,
    γ=1.0001, 
    θ = 1.0,# to be changed endogenously
    τl=0.0, # labor tax
    Tt_Y = 0.082, # lump-sum tax to GDP
    na=100,
    a_min=1e-9, 
    a_max=1000.0,
    curv_a=4.0,
    ny=5,
    ρy=0.9962034337990648, 
    σy=0.04383761259237633
);

In [3]:
economy_τl = Economy(
    KsY=10.26,
    α=0.36,
    δ=0.025,
    γ=1.0001, 
    θ = 1.0,# to be changed endogenously
    τl= 0.12513323082713207, # labor tax
    Tt_Y = 0., # lump-sum tax to GDP
    na=100,
    a_min=1e-9, 
    a_max=1000.0,
    curv_a=4.0,
    ny=5,
    ρy=0.9962034337990648, 
    σy=0.04383761259237633
);

## Computing the solution of the Aiyagari model

The function `steady` takes an input a set of economy parameters `economy` (as a `Economy` structure) and returns the solution of the Aiyagari model (as an `AiyagariSolution` structure). The `Rmin` and `Rmax` parameters are optional bounds for the dichotomy algorithm.
For further details on:
* the structures, refer to the notebook [`Structures`](./Structures.html);
* the Aiyagari solution, refer to the notebook [`SolveAiyagari`](./SolveAiyagari.html).

In [4]:
solution_Tt = steady(economy_Tt,Rmin=1.0086444151,Rmax=1.0086444153);

Markets clear! in: 1 iterations


In [4]:
solution_τl = steady(economy_τl,Rmin=1.0089195435,Rmax=1.0089195436);

Markets clear! in: 1 iterations


We check the consistency of `solution` (using `check_solution`) and provide a description of `solution` (using `describe_solution`). These two functions are defined in the notebook [`SolveAiyagari`](./SolveAiyagari.html).

In [5]:
@assert check_solution(solution_Tt, economy_Tt)
(print_dict ∘ describe_solution)(solution_Tt,economy_Tt)

Lτ = 1.00000000000298
01. Gini                               0.7085
02. Debt-to-GDP, B/Y                   0.0%
03. Public spending-to-GDP, G/Y        8.01%
04. Aggregate consumption-to-GDP, C/Y  65.24%
05. Capital-to-GDP, K/Y                267.5%
06. Investment-to-GDP, I/Y             26.75%
07. Transfers-to-GDP, Tt/Y             -8.01%
08. Aggregate labor supply, L          1.0
09. Average MPC                        0.22
10. Consumption tax-to-GDP             0.0%
11. Labor tax-to-GDP                   -0.0%
12. Capital tax-to-GDP                 0.0%
13. Total tax-to-GDP                   8.01%
14. Share of credit-constrained agents 20.46%


In [5]:
@assert check_solution(solution_τl, economy_τl)
(print_dict ∘ describe_solution)(solution_τl, economy_τl)

01. Gini                               0.7225
02. Debt-to-GDP, B/Y                   0.0%
03. Public spending-to-GDP, G/Y        8.01%
04. Aggregate consumption-to-GDP, C/Y  65.46%
05. Capital-to-GDP, K/Y                265.33%
06. Investment-to-GDP, I/Y             26.53%
07. Transfers-to-GDP, Tt/Y             -0.0%
08. Aggregate labor supply, L          1.0
09. Average MPC                        0.2434
10. Consumption tax-to-GDP             0.0%
11. Labor tax-to-GDP                   8.01%
12. Capital tax-to-GDP                 0.0%
13. Total tax-to-GDP                   8.01%
14. Share of credit-constrained agents 22.15%


## Aggregating the Aiyagari model to get the truncated model

We set the truncation length to 2 and compute the truncated model using the function `Project_plan`. We then check that the truncated model is consistent using the function `check_solution_trunc`. Both functions are defined in the notebook [`Truncation`](./Truncation.html).

In [11]:
# Setting the truncation length
for N = 2:6

# Computing the truncated model
    trunc = Truncate(N,solution_Tt,economy_Tt);  

    # Checking the consistency of the projected model
    @assert check_truncated_model(trunc,solution_Tt,economy_Tt)

    # Writing the results in the file todynare_Truncation.mat for running Dynare 
    Write_Dynare(trunc,solution_Tt,economy_Tt, outname="todynare_Truncation_N_$N.mat")
end

In [9]:
N = 1
"todynare_Truncation_N=$N.mat"

"todynare_Truncation_N=1.mat"

The model needs then to be run on Dynare.